In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
import nltk
nltk.download()
# In the NLTK Downloader window that pops up, click on the Models tab, 
# then click on punkt in the Identifier column, then click on the Download button.

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
scaler = StandardScaler()
svc = SVC()
Tfidf = TfidfVectorizer()
pipe = Pipeline(steps=[("Tfidf", Tfidf),("scaler", scaler), ("svc", svc)])

In [4]:
news_dataset = pd.read_csv('./fake-news/train.csv')
news_dataset.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [5]:
# Filling NaN fields with empty string
news_dataset = news_dataset.fillna('')

In [6]:
# Converting title and text columns to lowercase
news_dataset['title'] = news_dataset['title'].str.lower()
news_dataset.head()

,id,title,author,text,label
0,0,house dem aide: we didn’t even see comey’s let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"flynn: hillary clinton, big woman on campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,why the truth might get you fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 civilians killed in single us airstrike hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [8]:
news_dataset['content'] = news_dataset['content'].str.lower()
news_dataset.head()

,id,title,author,text,label,content
0,0,house dem aide: we didn’t even see comey’s let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrell lucus house dem aide: we didn’t even s...
1,1,"flynn: hillary clinton, big woman on campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"daniel j. flynn flynn: hillary clinton, big wo..."
2,2,why the truth might get you fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnews.com why the truth might get you...
3,3,15 civilians killed in single us airstrike hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss 15 civilians killed in single ...
4,4,iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jailed for fictio...


In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bverb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
port_stem = PorterStemmer()

In [12]:
# Function for removing stopwords and stemming words
# Could try Lemmatization as it's even better than stemming but it's more complicated
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [13]:
# Applying the stemming function
news_dataset['content'] = news_dataset['content'].apply(stemming)
news_dataset.head()

,id,title,author,text,label,content
0,0,house dem aide: we didn’t even see comey’s let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"flynn: hillary clinton, big woman on campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,why the truth might get you fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 civilians killed in single us airstrike hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...


In [14]:
print(news_dataset['content'].iloc[:1])

0    darrel lucu hous dem aid even see comey letter...
Name: content, dtype: object


In [15]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [16]:
print(X[22])

pam key rob reiner trump mental unstabl breitbart


In [21]:
# # converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=0)

In [19]:
# model = LogisticRegression()

In [23]:
pipe.fit(X_train, Y_train)

ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.

In [ ]:
pipe.score(X_test, Y_test)

In [ ]:
# # accuracy score on the training data
# X_train_prediction = pipe.predict(X_train)
# training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
# print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9864182692307693


In [ ]:
# # accuracy score on the test data
# X_test_prediction = pipe.predict(X_test)
# test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
# print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9800480769230769


In [ ]:
# import pickle
# with open('ben_test.pickle', 'wb') as picklefile:
#     pickle.dump(pipe, picklefile)

import joblib
filename = 'nlp_logistical_regression.sav'
joblib.dump(model, filename)